In [12]:
# Instalo las librerías que voy a utilizar

!pip install requests

!pip install openpyxl


[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached openpyxl-3.1.2-py2.py3-none-any.whl (249 kB)
  Using cached et_xmlfile-1.1.0-py3-none-any.whl (4.7 kB)



[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
#Importo las librerías que voy a utilizar

import pandas as pd
import requests as rq
import os


In [14]:
# Creo las listas con el nombre de los documentos

lista_archivos = ["2000", "2001", "2002", "2003", "2004", "2005", "2006", "2007", "2008", "2009", "2010", "2011",
                  "2012", "2013", "2014", "2015", "2016", "2017", "2018", "2019", "2020", "2021", "2022"]


In [177]:
# Creo la función que me permitira cargar los datos de las hojas CONSUMOXCAPITA

def carga_consumoxcapita(nombre_archivo):
    base_url = "https://raw.githubusercontent.com"
    usuario_git = "DMorgon"
    repositorio = "portafolios/main/alimentacion/datos_origen"
    ruta_archivo = f"{base_url}/{usuario_git}/{repositorio}/{nombre_archivo}.xlsx"
    response = rq.get(ruta_archivo)
    if int(nombre_archivo) <= 2019:
        df_consumoxcapita = pd.read_excel(response.content, sheet_name=4, header=2, engine="openpyxl")
        df_consumoxcapita["ANALISIS"] = "Consumo per capita"
    else:
        df_consumoxcapita = pd.read_excel(response.content, sheet_name=5, header=2, engine="openpyxl")
        df_consumoxcapita["ANALISIS"] = "Consumo per capita"
    return df_consumoxcapita


In [147]:
# Creo la función que me permitira cargar los datos de las hojas GASTOXCAPITA

def carga_gastoxcapita(nombre_archivo):
    base_url = "https://raw.githubusercontent.com"
    usuario_git = "DMorgon"
    repositorio = "portafolios/main/alimentacion/datos_origen"
    ruta_archivo = f"{base_url}/{usuario_git}/{repositorio}/{nombre_archivo}.xlsx"
    response = rq.get(ruta_archivo)
    if int(nombre_archivo) <= 2019:
        df_gastoxcapita = pd.read_excel(response.content, sheet_name=5, header=2, engine="openpyxl")
        df_gastoxcapita["ANALISIS"] = "Gasto per cápita"
    else:
        df_gastoxcapita = pd.read_excel(response.content, sheet_name=6, header=2, engine="openpyxl")
        df_gastoxcapita["ANALISIS"] = "Gasto per cápita"
    return df_gastoxcapita


In [148]:
# Creo la función que me permitira cargar los datos de las hojas PRECIO

def carga_precio(nombre_archivo):
    base_url = "https://raw.githubusercontent.com"
    usuario_git = "DMorgon"
    repositorio = "portafolios/main/alimentacion/datos_origen"
    ruta_archivo = f"{base_url}/{usuario_git}/{repositorio}/{nombre_archivo}.xlsx"
    response = rq.get(ruta_archivo)
    if int(nombre_archivo) <= 2019:
        df_precio = pd.read_excel(response.content, sheet_name=6, header=2, engine="openpyxl")
        df_precio["ANALISIS"] = "Precio"
    else:
        df_precio = pd.read_excel(response.content, sheet_name=4, header=2, engine="openpyxl")
        df_precio["ANALISIS"] = "Precio"
    return df_precio


In [178]:
# Cargo los datos correspondientes con las hojas consumoxcapita de cada archivo.

lista_df_consumoxcapita = []

for archivo in lista_archivos:
    df = carga_consumoxcapita(archivo)
    lista_df_consumoxcapita.append(df)
    

In [150]:
# Cargo los datos correspondientes con las hojas gastoxcapita de cada archivo.

lista_df_gastoxcapita = []

for archivo in lista_archivos:
    df = carga_gastoxcapita(archivo)
    lista_df_gastoxcapita.append(df)
    

In [151]:
# Uno los marcos de datos según el año y los incluyo en la lista_df

lista_df = []

for i in range(0, 23, 1):
    df = pd.concat([lista_df_consumoxcapita[i], lista_df_gastoxcapita[i]], ignore_index=True)
    lista_df.append(df)
    

## ANÁLISIS EXPLORATORIO DE LOS DATOS (EDA)



In [154]:
# Realizo EDA en cada marco de datos

for i, df in enumerate(lista_df):
    tabla_nombre = f"df_{2000 + i}"
    print("Nombre de la tabla de datos:", tabla_nombre)
    print(df.info())
    if df.isna().any().any():
        print("Hay valores NA en la tabla de datos" + "\n")
    else:
        print("No hay valores NA en la tabla de datos" + "\n")


Nombre de la tabla de datos: df_2000
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 668 entries, 0 to 667
Data columns (total 20 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Unnamed: 0          668 non-null    object 
 1   .TOTAL ESPAÑA       666 non-null    float64
 2   CATALUÑA            666 non-null    float64
 3   ARAGON              654 non-null    float64
 4   BALEARES            658 non-null    float64
 5   VALENCIA            664 non-null    float64
 6   MURCIA              660 non-null    float64
 7   ANDALUCIA           664 non-null    float64
 8   MADRID              664 non-null    float64
 9   CASTILLA-LA MANCHA  660 non-null    float64
 10  EXTREMADURA         654 non-null    float64
 11  CASTILLA Y LEON     664 non-null    float64
 12  GALICIA             664 non-null    float64
 13  ASTURIAS            662 non-null    float64
 14  CANTABRIA           660 non-null    float64
 15  PAIS VASCO          

Revisando la información de cada marco de datos, encontramos las siguientes anomalías:

- En los marcos dedatos de los años 2000, 2001, 2002 y 2003 existen valores NA

- Los marcos de datos referentes a los años 2000, 2001, 2002, 2003, 2019, 2020, 2021 y 2022 tienen 20 columnas mientras que el resto de las tablas tienen 28. Hay variables que no son necesarias, para el análisis, con lo que pueden ser eliminadas.

- Existen marcos de datos con el nombre de las variables de ciertas comunidades autónomas diferentes.

- La variable de las categorías de los alimentos se denomina Unamed: 0.

In [155]:
# Elimino las variables correspondientes a los territorios que no voy a utilizar

columnas_eliminar_a = ["T.ESPAÑA", "T.ANDALUCIA", "NORESTE", "LEVANTE", "CENTRO-SUR", "CASTILLA LEON", "NOROESTE", "NORTE",
                       "T.CANARIAS"]

columnas_eliminar_b = ["T.ESPAÑA", "NORESTE", "LEVANTE", "ANDALUCÍA", "CENTRO-SUR", "CASTILLA Y LEÓN", "NOROESTE", "NORTE",
                       "T.CANARIAS"]


for i, df in enumerate(lista_df):
    if i in [0, 1, 2, 3]:
        df.drop(columns=".TOTAL ESPAÑA", errors="ignore", inplace=True)
    if i in [4, 5, 6, 7, 8, 9, 10, 11, 12]:
        df.drop(columns=columnas_eliminar_a, errors="ignore", inplace=True)
    if i in [13, 14, 15, 16, 17, 18]:
        df.drop(columns=columnas_eliminar_b, errors="ignore", inplace=True)  
    else:
        df.drop(columns="T.ESPAÑA", errors="ignore", inplace=True)


In [156]:
# Realizo EDA en cada marco de datos

for i, df in enumerate(lista_df):
    tabla_nombre = f"df_{2000 + i}"
    print("Nombre de la tabla de datos:", tabla_nombre)
    print(df.info())
    if df.isna().any().any():
        print("Hay valores NA en la tabla de datos" + "\n")
    else:
        print("No hay valores NA en la tabla de datos" + "\n")


Nombre de la tabla de datos: df_2000
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 668 entries, 0 to 667
Data columns (total 19 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Unnamed: 0          668 non-null    object 
 1   CATALUÑA            666 non-null    float64
 2   ARAGON              654 non-null    float64
 3   BALEARES            658 non-null    float64
 4   VALENCIA            664 non-null    float64
 5   MURCIA              660 non-null    float64
 6   ANDALUCIA           664 non-null    float64
 7   MADRID              664 non-null    float64
 8   CASTILLA-LA MANCHA  660 non-null    float64
 9   EXTREMADURA         654 non-null    float64
 10  CASTILLA Y LEON     664 non-null    float64
 11  GALICIA             664 non-null    float64
 12  ASTURIAS            662 non-null    float64
 13  CANTABRIA           660 non-null    float64
 14  PAIS VASCO          664 non-null    float64
 15  RIOJA               

In [157]:
# Corrijo el nombre de las variables correspondientes a las comunidades autónomas.

for df in lista_df:
    df.columns = df.columns.str.replace("Unnamed: 0", "CATEGORIAS", regex=False)
    df.columns = df.columns.str.replace("CATALUÑA", "Cataluña", regex=False)
    df.columns = df.columns.str.replace("ARAGON", "Aragón", regex=False)
    df.columns = df.columns.str.replace("ARAGÓN", "Aragón", regex=False)
    df.columns = df.columns.str.replace("BALEARES", "Baleares", regex=False)
    df.columns = df.columns.str.replace("ILLES BALEARS", "Baleares", regex=False)
    df.columns = df.columns.str.replace("VALENCIA", "Valencia", regex=False)
    df.columns = df.columns.str.replace("COMUNITAT VALENCIANA", "Valencia", regex=False)
    df.columns = df.columns.str.replace("COMUNITAT ValenciaNA", "Valencia", regex=False)
    df.columns = df.columns.str.replace("MURCIA", "Murcia", regex=False)
    df.columns = df.columns.str.replace("REGIÓN DE MURCIA", "Murcia", regex=False)
    df.columns = df.columns.str.replace("REGIÓN DE Murcia", "Murcia", regex=False)
    df.columns = df.columns.str.replace("MADRID", "Madrid", regex=False)
    df.columns = df.columns.str.replace("COMUNIDAD DE Madrid", "Madrid", regex=False)
    df.columns = df.columns.str.replace("COMUNIDAD DE MADRID", "Madrid", regex=False)
    df.columns = df.columns.str.replace("CASTILLA-LA MANCHA", "Castilla La Mancha", regex=False)
    df.columns = df.columns.str.replace("CASTILLA LA MANCHA", "Castilla La Mancha", regex=False)
    df.columns = df.columns.str.replace("CASTILLA - LA MANCHA", "Castilla La Mancha", regex=False)
    df.columns = df.columns.str.replace("EXTREMADURA", "Extremadura", regex=False)
    df.columns = df.columns.str.replace("GALICIA", "Galicia", regex=False)
    df.columns = df.columns.str.replace("ASTURIAS", "Asturias", regex=False)
    df.columns = df.columns.str.replace("PRINCIPADO DE ASTURIAS", "Asturias", regex=False)
    df.columns = df.columns.str.replace("PRINCIPADO DE Asturias", "Asturias", regex=False)
    df.columns = df.columns.str.replace("CANTABRIA", "Cantabria", regex=False)
    df.columns = df.columns.str.replace("PAIS VASCO", "País Vasco", regex=False)
    df.columns = df.columns.str.replace("RIOJA", "La Rioja", regex=False)
    df.columns = df.columns.str.replace("LA RIOJA", "La Rioja", regex=False)
    df.columns = df.columns.str.replace("NAVARRA", "Navarra", regex=False)
    df.columns = df.columns.str.replace("C. FORAL DE NAVARRA", "Navarra", regex=False)
    df.columns = df.columns.str.replace("C. FORAL DE Navarra", "Navarra", regex=False)
    df.columns = df.columns.str.replace("CANARIAS", "Canarias", regex=False)
    df.columns = df.columns.str.replace("ANDALUCIA", "Andalucía", regex=False)
    df.columns = df.columns.str.replace("ANDALUCÍA", "Andalucía", regex=False)
    df.columns = df.columns.str.replace("CASTILLA Y LEON", "Castilla y León", regex=False)
    df.columns = df.columns.str.replace("CASTILLA Y LEÓN", "Castilla y León", regex=False)
    df.loc[:, sorted(df.columns)] = df.loc[:, df.columns]


In [158]:
# Realizo EDA en cada marco de datos

for i, df in enumerate(lista_df):
    tabla_nombre = f"df_{2000 + i}"
    print("Nombre de la tabla de datos:", tabla_nombre)
    print(df.info())
    if df.isna().any().any():
        print("Hay valores NA en la tabla de datos" + "\n")
    else:
        print("No hay valores NA en la tabla de datos" + "\n")


Nombre de la tabla de datos: df_2000
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 668 entries, 0 to 667
Data columns (total 19 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   CATEGORIAS          668 non-null    object 
 1   Cataluña            666 non-null    float64
 2   Aragón              654 non-null    float64
 3   Baleares            658 non-null    float64
 4   Valencia            664 non-null    float64
 5   Murcia              660 non-null    float64
 6   Andalucía           664 non-null    float64
 7   Madrid              664 non-null    float64
 8   Castilla La Mancha  660 non-null    float64
 9   Extremadura         654 non-null    float64
 10  Castilla y León     664 non-null    float64
 11  Galicia             664 non-null    float64
 12  Asturias            662 non-null    float64
 13  Cantabria           660 non-null    float64
 14  País Vasco          664 non-null    float64
 15  La Rioja            

In [159]:
# Añado la columna AÑO con los valores correspondiente al año del marco de datos

for i, df in enumerate(lista_df):
    df["AÑO"] = int(lista_archivos[i])


In [160]:
# Uno todas las tablas en una sola df_total

df_total = pd.concat(objs=lista_df, axis=0)


In [161]:
# Realizó un filtro en df_total para sólo quedarme con las categorías de alimentos que voy a utilizar

lista_categoria = ["T.HUEVOS KGS", "MIEL", "TOTAL CARNE", "TOTAL PESCA", "TOTAL LECHE LIQUIDA", "TOTAL OTRAS LECHES",
                   "DERIVADOS LACTEOS", "PAN", "BOLL.PAST.GALLET.CERE", "PRODUCTOS NAVIDEÑOS",
                   "CHOCOLATES/CACAOS/SUC", "ARROZ", "TOTAL PASTAS", "AZUCAR", "EDULCORANTES", "LEGUMBRES",
                   "TOTAL ACEITE", "MARGARINA", "ACEITUNAS", "BEBIDAS DERIVADAS VI", "TOTAL VINOS", "CERVEZAS",
                   "SIDRAS", "T.BEBIDAS ESPIRITUOSA", "VINAGRE", "TOTAL ZOMO Y NECTAR", "TOTAL PATATAS",
                   "T.HORTALIZAS FRESCAS", "T.FRUTAS FRESCAS", "FRUTOS SECOS", "T.FRUTA&HORTA.TRANSF",
                   "PLATOS PREPARADOS", "CAFES E INFUSIONES", "CALDOS", "SALSAS", "AGUA DE BEBIDA ENVAS.",
                   "GASEOSAS Y BEBID.REFR", "BASES PIZZAS&MASAS HO", "HARINAS Y SEMOLAS", "ENCURTIDOS",
                   "ESPECIAS Y CONDIMENTO", "SAL", "OTROS PROD.EN PESO", "OTROS PROD.EN VOLUMEN", "ALGAS"]

df_total = df_total[df_total["CATEGORIAS"].isin(lista_categoria)]


In [162]:
# Corrijo el nombre de la categoría de los alimentos


nuevas_categorias = {"T.HUEVOS KGS": "Huevos", "MIEL": "Miel", "TOTAL CARNE": "Carne",
                     "TOTAL PESCA": "Pescados y mariscos", "TOTAL LECHE LIQUIDA": "Leche líquida",
                     "TOTAL OTRAS LECHES": "Otras leches", "DERIVADOS LACTEOS": "Derivados lacteos", "PAN": "Pan",
                     "BOLL.PAST.GALLET.CERE": "Bolleria, pastelería, galletas y cereales",
                     "PRODUCTOS NAVIDEÑOS": "Productos navideños", "CHOCOLATES/CACAOS/SUC": "Chocolates",
                     "ARROZ": "Arroz", "TOTAL PASTAS": "Pastas", "AZUCAR": "Azucar", "EDULCORANTES": "Edulcorante",
                     "LEGUMBRES": "Legumbres", "TOTAL ACEITE": "Aceites", "MARGARINA": "Margarinas",
                     "ACEITUNAS": "Aceitunas", "BEBIDAS DERIVADAS VI": "Bebidas derivadas del vino",
                     "TOTAL VINOS": "Vinos", "CERVEZAS": "Cervezas", "SIDRAS": "Sidras",
                     "T.BEBIDAS ESPIRITUOSA": "Bebidas espirituosas", "VINAGRE": "Vinagres",
                     "TOTAL ZOMO Y NECTAR": "Zumos y nectares", "TOTAL PATATAS": "Patatas",
                     "T.HORTALIZAS FRESCAS": "Hortalizas frescas", "T.FRUTAS FRESCAS": "Frutas frescas",
                     "FRUTOS SECOS": "Frutos secos", "T.FRUTA&HORTA.TRANSF": "Frutas y hortalizas transformadas",
                     "PLATOS PREPARADOS": "Platos preparados", "CAFES E INFUSIONES": "Cafes e infusiones",
                     "CALDOS": "Caldos", "SALSAS": "Salsas", "AGUA DE BEBIDA ENVAS.": "Aguas envasadas",
                     "GASEOSAS Y BEBID.REFR": "Gaseosas y bebidas refrescantes", "BASES PIZZAS&MASAS HO": "Masas",
                     "HARINAS Y SEMOLAS": "Harinas y semolas", "ENCURTIDOS": "Encurtidos",
                     "ESPECIAS Y CONDIMENTO": "Especias", "SAL": "Sal", "OTROS PROD.EN PESO": "Otros productos en peso",
                     "OTROS PROD.EN VOLUMEN": "Otros productos en volumen", "ALGAS": "Algas"}

df_total["CATEGORIAS"] = df_total["CATEGORIAS"].replace(nuevas_categorias)


In [163]:
# Transformo el formato de ancho a largo, manteniendo las columnas "AÑO", "CATEGORIAS" y "ANALISIS" como identificadores


df_total = df_total.melt(id_vars=["AÑO", "CATEGORIAS", "ANALISIS"], var_name="COMUNIDAD AUTONOMA", value_name="CANTIDAD")


In [164]:
# Realizo EDA en df_total

print(df_total.info())
if df_total.isna().any().any():
    print("Hay valores NA en la tabla de datos" + "\n")
else:
    print("No hay valores NA en la tabla de datos" + "\n")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32760 entries, 0 to 32759
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   AÑO                 32760 non-null  int64  
 1   CATEGORIAS          32760 non-null  object 
 2   ANALISIS            32760 non-null  object 
 3   COMUNIDAD AUTONOMA  32760 non-null  object 
 4   CANTIDAD            30940 non-null  float64
dtypes: float64(1), int64(1), object(3)
memory usage: 1.2+ MB
None
Hay valores NA en la tabla de datos



Según se ha podido comprobar en EDA, existen valores NA. A continuación localizaré en que registros existen dichos valores

In [176]:
# Localizo en que categorías de alimentos estan los valores NA

valores_faltantes = df_total.isna()

registros_con_faltantes = df_total[valores_faltantes.any(axis=1)]

pd.set_option('display.max_rows', None)

#print(registros_con_faltantes)

print(registros_con_faltantes["ANALISIS"].unique())
print(registros_con_faltantes[registros_con_faltantes["ANALISIS"] == "Gasto per cápita"])
#print(registros_con_faltantes["AÑO"].unique())


['Consumo per capita' 'Gasto per cápita']
        AÑO                                 CATEGORIAS          ANALISIS  \
25769  2004                                       Miel  Gasto per cápita   
25770  2004                                      Carne  Gasto per cápita   
25771  2004                        Pescados y mariscos  Gasto per cápita   
25772  2004                              Leche líquida  Gasto per cápita   
25773  2004                               Otras leches  Gasto per cápita   
25774  2004                          Derivados lacteos  Gasto per cápita   
25775  2004                                        Pan  Gasto per cápita   
25776  2004  Bolleria, pastelería, galletas y cereales  Gasto per cápita   
25777  2004                        Productos navideños  Gasto per cápita   
25778  2004                                 Chocolates  Gasto per cápita   
25779  2004                                      Arroz  Gasto per cápita   
25780  2004                                   